In [1]:
import tensorflow as tf
import csv
import os
import numpy as np
from tensorflow import keras as tfk
from keras import layers as tfkl
import sys
from data_creation import *

# Import constants and catch any import errors
from link_to_constants import *
link_constants()
try:
    from constants import * # type: ignore
    print("Successfully imported constants.")
except ImportError as e:
    print(f"Error importing constants: {e}")

Current directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\neural_network
Parent directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy
Data gen and processing directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\data_gen_and_processing
Successfully imported constants.
Current directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\neural_network
Parent directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy
Data gen and processing directory: c:\Users\marco\Documents\GitHub\Tesi\Codice_carla\final - Copy\data_gen_and_processing
Successfully imported constants.


In [7]:
complete_grid_maps = []
complete_grid_maps_BB = []

# Load sensor1
grid_maps = generate_grid_map(LIDAR_1_GRID_DIRECTORY) # type: ignore
print("-------------------------")
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_1_GRID_DIRECTORY) # type: ignore

grid_maps.append(complete_grid_maps)
grid_maps_BB.append(complete_grid_maps_BB)

# Load sensor2
grid_maps = generate_grid_map(LIDAR_2_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_2_GRID_DIRECTORY) # type: ignore

grid_maps.append(complete_grid_maps)
grid_maps_BB.append(complete_grid_maps_BB)

# Load sensor3
grid_maps = generate_grid_map(LIDAR_3_GRID_DIRECTORY) # type: ignore
grid_maps_BB = generate_grid_map_BB(NEW_POSITIONS_LIDAR_3_GRID_DIRECTORY) # type: ignore

complete_grid_maps.append(grid_maps)
complete_grid_maps_BB.append(grid_maps_BB)

['20250118_102725_853040_169399.csv', '20250118_102725_975855_169402.csv', '20250118_102726_071063_169404.csv', '20250118_102726_162930_169406.csv', '20250118_102726_243632_169408.csv', '20250118_102726_321813_169410.csv', '20250118_102726_399399_169412.csv', '20250118_102726_476882_169414.csv', '20250118_102726_558496_169416.csv', '20250118_102726_635325_169418.csv', '20250118_102726_713490_169420.csv', '20250118_102726_789580_169422.csv', '20250118_102726_868563_169424.csv', '20250118_102726_947714_169426.csv', '20250118_102727_022729_169428.csv', '20250118_102727_100899_169430.csv', '20250118_102727_181769_169432.csv', '20250118_102727_260511_169434.csv', '20250118_102727_348745_169436.csv', '20250118_102727_432632_169438.csv', '20250118_102727_522459_169440.csv', '20250118_102727_614992_169442.csv', '20250118_102727_710507_169444.csv', '20250118_102727_803280_169446.csv', '20250118_102727_882485_169448.csv', '20250118_102727_958714_169450.csv', '20250118_102728_037598_169452.csv', 

ValueError: could not convert string to float: 'vertice_1x'

In [7]:
print(len(complete_grid_maps))

1


In [4]:
print(complete_grid_maps)

[[]]


In [5]:
 # Split the data
X_train_val, X_test, y_train_val, y_test = split_data(complete_grid_maps, complete_grid_maps_BB)
X_train, X_val, y_train, y_val = split_data(complete_grid_maps, complete_grid_maps_BB, test_size = len(X_test)) # Esure that val and test set have the same lenght

ValueError: With n_samples=1, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Initialize the scaler
scaler = MinMaxScaler()

# Normalize the data
X_train = scaler.fit_transform(X_train.reshape(-1, X_train.shape[-1])).reshape(X_train.shape)
X_val = scaler.transform(X_val.reshape(-1, X_val.shape[-1])).reshape(X_val.shape)
X_test = scaler.transform(X_test.reshape(-1, X_test.shape[-1])).reshape(X_test.shape)

y_train = scaler.fit_transform(y_train.reshape(-1, y_train.shape[-1])).reshape(y_train.shape)
y_val = scaler.transform(y_val.reshape(-1, y_val.shape[-1])).reshape(y_val.shape)
y_test = scaler.transform(y_test.reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

In [ ]:
shape_input = (X_RANGE,Y_RANGE,1) # type: ignore
    
# Input layer for the 2.5D grid map
grid_map_input = tfkl.Input(shape_input, name='grid_map_input')

# Encoder
x = tfkl.Conv2D(32, (3, 3), activation='relu', padding='same')(grid_map_input)
x = tfkl.MaxPooling2D((2, 2), padding='same')(x)
x = tfkl.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tfkl.MaxPooling2D((2, 2), padding='same')(x)
x = tfkl.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = tfkl.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = tfkl.Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = tfkl.UpSampling2D((2, 2))(x)
x = tfkl.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tfkl.UpSampling2D((2, 2))(x)
x = tfkl.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tfkl.UpSampling2D((2, 2))(x)
decoded = tfkl.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Create the model
model = tfkl.Model(inputs=grid_map_input, outputs=decoded)

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Print the model summary
model.summary()